In [2]:
#1.kutuphaneler
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#2.veri onisleme
#2.1.veri yukleme
veriler = pd.read_csv('veriler.csv')
veriler['yas'] = veriler['yas'].fillna(28)
#pd.read_csv("veriler.csv")
#test
#print(veriler)
#------------------------------
#encoder: Kategorik -> Numeric
ulke = veriler.iloc[:,0:1].values
Yas = veriler.iloc[:,1:4].values
#print(ulke)

from sklearn import preprocessing


le = preprocessing.LabelEncoder() ####0-1-2 şeklinde ayırıyor 3 farklı ülkeyi
ulke[:,0] = le.fit_transform(veriler.iloc[:,0])

#print(ulke)


ohe = preprocessing.OneHotEncoder()
ulke = ohe.fit_transform(ulke).toarray() ####3 farklı ülkeyi sadece 0-1 kullanarak şekillendiriyor. 
#print(ulke)
#---------------------------------------
#encoder: Kategorik -> Numeric
c = veriler.iloc[:,-1:].values
#print(c)


from sklearn import preprocessing

le = preprocessing.LabelEncoder() #0-1 dönüşümünü yapıyor

c[:,-1] = le.fit_transform(veriler.iloc[:,-1])

#print(c)


ohe = preprocessing.OneHotEncoder() #dummy variable için yazılmış bir kod 
c = ohe.fit_transform(c).toarray()
#print(c)
#-----------------------------------------------

#numpy dizileri dataframe donusumu
sonuc = pd.DataFrame(data=ulke, index = range(22), columns = ['fr','tr','us'])
#print(sonuc)

sonuc2 = pd.DataFrame(data=Yas, index = range(22), columns = ['boy','kilo','yas'])
#print(sonuc2)

cinsiyet = veriler.iloc[:,-1].values
#print(cinsiyet)

sonuc3 = pd.DataFrame(data = c[:,:1] , index = range(22), columns = ['cinsiyet'])
#print(sonuc3)



#dataframe birlestirme islemi
s=pd.concat([sonuc,sonuc2], axis=1)
#print(s)

s2=pd.concat([s,sonuc3], axis=1)
#print(s2)

#verilerin egitim ve test icin bolunmesi
from sklearn.model_selection import train_test_split

x_train, x_test,y_train,y_test = train_test_split(s,sonuc3,test_size=0.33, random_state=0)

from sklearn.linear_model import LinearRegression

regressor = LinearRegression() #classdan bir obje tanımladık 
regressor.fit(x_train,y_train) # y_traini öğren dedim, x_traini kullanarak
y_pred = regressor.predict(x_test)
#boy tahmini yapacağımi o yüzden x_train de boy değerleri olmayacak, y_trainde bulunacak 
boy = s2.iloc[:,3:4].values
#print(boy)
sol = s2.iloc[:,:3]
sag= s2.iloc[:,4:]
veri = pd.concat([sol,sag], axis=1) #boy olmadan oluşturulan df 

#bağımlı değişkenim boy, o yüzden verileri kullanarak boy u tahmin et diyorum 
x_train, x_test,y_train,y_test = train_test_split(veri,boy,test_size=0.33, random_state=0)


from sklearn.linear_model import LinearRegression
r2 = LinearRegression() 
r2.fit(x_train,y_train) 
y_pred = r2.predict(x_test)

y_pred





import statsmodels.api as sm # modelin ve modeldeki değişkenlerin başarısı ile ilgili bir kütüphane 
#geri eleme yöntemini kullanıyoruz. hangi değişkenin p valuesi yüksekse onu atmaya çalışacağız
#denklemimizde olan en başta olan b0 sabit değeri kodlarda yok. şuan yapmaya çalıştığımı o 1leri eklemek 
X = np.append(arr=np.ones((22,1)).astype(int), values=veri, axis=1)

X_l = veri.iloc[:,[0,1,2,3,4,5]].values # aslında kolonların hepsini aldı, ama böyle yazmasının sebebi daha sonradan elemek istediği kolonları silecek
model = sm.OLS(boy,X_l).fit() # boy kolonunu merak ediyoruz ve bütün kolonları verdik. boy bağımlı değişken 
#OLS istatiksel değerlerin oluşumunu sağlıyor
print(model.summary())  #sonuçlara baktığımızda x5(yani beşinci colon) in p valuesi 0.05 den yüksek
# bu sebeple onu elemek zorundayız. çünkü kullandığımız eleme algoritması bize bunu söylüyor 


X_l = veri.iloc[:,[0,1,2,3,5]].values 
model = sm.OLS(boy,X_l).fit() 
print(model.summary()) 
# 5.veri kalabilir. 0.031 değeri 0.05 değerinden küçük olduğu için kalabilir. ancak silmek isterseniz silebilirsiniz. 


X_l = veri.iloc[:,[0,1,2,3]].values 
model = sm.OLS(boy,X_l).fit() 
print(model.summary()) #P>|t| Değeri 0.000 oldu

 







                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.886
Model:                            OLS   Adj. R-squared:                  0.850
Method:                 Least Squares   F-statistic:                     24.84
Date:                Wed, 07 Jun 2023   Prob (F-statistic):           5.19e-07
Time:                        14:49:57   Log-Likelihood:                -73.892
No. Observations:                  22   AIC:                             159.8
Df Residuals:                      16   BIC:                             166.3
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1           113.6779      8.074     14.080      0.0

In [81]:
X

array([[  1.,   0.,   1.,   0.,  30.,  10.,   1.],
       [  1.,   0.,   1.,   0.,  36.,  11.,   1.],
       [  1.,   0.,   1.,   0.,  34.,  10.,   0.],
       [  1.,   0.,   1.,   0.,  30.,   9.,   0.],
       [  1.,   0.,   1.,   0.,  38.,  12.,   1.],
       [  1.,   0.,   1.,   0.,  90.,  30.,   1.],
       [  1.,   0.,   1.,   0.,  80.,  25.,   1.],
       [  1.,   0.,   1.,   0.,  90.,  35.,   1.],
       [  1.,   0.,   1.,   0.,  60.,  22.,   0.],
       [  1.,   0.,   0.,   1., 105.,  33.,   1.],
       [  1.,   0.,   0.,   1.,  55.,  27.,   0.],
       [  1.,   0.,   0.,   1.,  50.,  44.,   0.],
       [  1.,   0.,   0.,   1.,  58.,  28.,   0.],
       [  1.,   0.,   0.,   1.,  59.,  41.,   0.],
       [  1.,   0.,   0.,   1.,  62.,  55.,   0.],
       [  1.,   1.,   0.,   0.,  70.,  47.,   1.],
       [  1.,   1.,   0.,   0.,  90.,  28.,   1.],
       [  1.,   1.,   0.,   0.,  80.,  27.,   1.],
       [  1.,   1.,   0.,   0.,  88.,  28.,   1.],
       [  1.,   1.,   0.,   0.,

In [75]:
X_l

array([[  0.,   1.,   0.,  30.,  10.,   1.],
       [  0.,   1.,   0.,  36.,  11.,   1.],
       [  0.,   1.,   0.,  34.,  10.,   0.],
       [  0.,   1.,   0.,  30.,   9.,   0.],
       [  0.,   1.,   0.,  38.,  12.,   1.],
       [  0.,   1.,   0.,  90.,  30.,   1.],
       [  0.,   1.,   0.,  80.,  25.,   1.],
       [  0.,   1.,   0.,  90.,  35.,   1.],
       [  0.,   1.,   0.,  60.,  22.,   0.],
       [  0.,   0.,   1., 105.,  33.,   1.],
       [  0.,   0.,   1.,  55.,  27.,   0.],
       [  0.,   0.,   1.,  50.,  44.,   0.],
       [  0.,   0.,   1.,  58.,  28.,   0.],
       [  0.,   0.,   1.,  59.,  41.,   0.],
       [  0.,   0.,   1.,  62.,  55.,   0.],
       [  1.,   0.,   0.,  70.,  47.,   1.],
       [  1.,   0.,   0.,  90.,  28.,   1.],
       [  1.,   0.,   0.,  80.,  27.,   1.],
       [  1.,   0.,   0.,  88.,  28.,   1.],
       [  1.,   0.,   0.,  40.,  29.,   0.],
       [  1.,   0.,   0.,  66.,  32.,   0.],
       [  1.,   0.,   0.,  56.,  42.,   0.]])

In [3]:
s

,fr,tr,us,boy,kilo,yas
0,0.0,1.0,0.0,130.0,30.0,10.0
1,0.0,1.0,0.0,125.0,36.0,11.0
2,0.0,1.0,0.0,135.0,34.0,10.0
3,0.0,1.0,0.0,133.0,30.0,9.0
4,0.0,1.0,0.0,129.0,38.0,12.0
5,0.0,1.0,0.0,180.0,90.0,30.0
6,0.0,1.0,0.0,190.0,80.0,25.0
7,0.0,1.0,0.0,175.0,90.0,35.0
8,0.0,1.0,0.0,177.0,60.0,22.0
9,0.0,0.0,1.0,185.0,105.0,33.0


In [5]:
sonuc3

,cinsiyet
0,1.0
1,1.0
2,0.0
3,0.0
4,1.0
5,1.0
6,1.0
7,1.0
8,0.0
9,1.0


In [6]:
veri

,fr,tr,us,kilo,yas,cinsiyet
0,0.0,1.0,0.0,30.0,10.0,1.0
1,0.0,1.0,0.0,36.0,11.0,1.0
2,0.0,1.0,0.0,34.0,10.0,0.0
3,0.0,1.0,0.0,30.0,9.0,0.0
4,0.0,1.0,0.0,38.0,12.0,1.0
5,0.0,1.0,0.0,90.0,30.0,1.0
6,0.0,1.0,0.0,80.0,25.0,1.0
7,0.0,1.0,0.0,90.0,35.0,1.0
8,0.0,1.0,0.0,60.0,22.0,0.0
9,0.0,0.0,1.0,105.0,33.0,1.0
